# 빅데이터 프로젝트 2일차 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import matplotlib
# 한글출력
import matplotlib.font_manager as fm

# font_location = '/usr/share/fonts/truetype/nanum/NanumSquareB.ttf'
# font_name = fm.FontProperties(fname = font_location).get_name()
# matplotlib.rc('font',family = font_name)

mfile = pd.read_csv('member_data.csv', encoding = 'utf-8')
#mfile.head()
dt1 = pd.read_csv('product_data_01.csv')
dt2 = pd.read_csv('product_data_02.csv')
dt3 = pd.read_csv('product_data_03.csv')
dt4 = pd.read_csv('product_data_04.csv')


## 오전,오후,저녁(12시, 5시) 방문율

In [2]:
dt1

,공급일자,회원번호,조합원상태,물품대분류,물품중분류,물품소분류,물품명,구매수량,구매금액,구매매장,반품_원거래일자,구매시각
0,2018-01-02,272369856,정상회원,과실,과일,사과,사과/유(1.5kg),1.0,22207.0,매장C,NaN,10:04
1,2018-01-02,1476143616,정상회원,간식,빵,"식빵, 식사대용",아침빵(240g/8개)/매장,1.0,4403.0,매장C,NaN,10:09
2,2018-01-02,743192512,정상회원,채소,양념채소,생강,생강(300g),1.0,7083.0,매장C,NaN,10:12
3,2018-01-02,176912064,정상회원,축산물,유제품,우유/산양유,유기농저지방우유(종이팩/900ml),1.0,7849.0,매장C,NaN,10:23
4,2018-01-02,87592064,정상회원,양념/가루,가루,양념가루,카레(매운맛: 100g),3.0,15507.0,매장C,NaN,10:29
...,...,...,...,...,...,...,...,...,...,...,...,...
123186,2018-10-31,67602944,정상회원,채소,잎/줄기채소,브로컬리,브로컬리(250g),2.0,10146.0,매장C,NaN,10:10
123187,2018-10-31,1285215040,정상회원,양념/가루,가루,프리믹스,핫케이크가루(600g),1.0,8041.0,매장C,NaN,13:56
123188,2018-10-31,335544384,정상회원,축산물,알,유정란,동물복지유정란(15알/국내산),1.0,10146.0,매장C,NaN,17:06
123189,2018-10-31,744100096,정상회원,축산물,돼지,무항생제돼지,얇은삼겹살(500g/국내산),2.0,48243.0,매장C,NaN,17:35


In [ ]:
dt2

In [16]:
dt1['구매수량'].value_counts()

 1.00     104862
 2.00      12326
 3.00       2125
-1.00       1650
 4.00        623
 0.10        324
 5.00        322
-2.00        197
 0.50        148
 0.20        143
 6.00        129
 0.30         55
 7.00         55
 10.00        40
-3.00         36
 0.00         25
 8.00         23
 0.40         15
 0.25         13
-4.00         12
 12.00         6
 9.00          5
-5.00          5
 20.00         5
 0.60          4
-0.50          4
 11.00         4
-0.10          3
 14.00         2
 0.33          2
-30.00         2
-7.00          2
 1.10          2
-10.00         2
 30.00         1
 0.18          1
 15.00         1
 0.80          1
 21.00         1
 17.00         1
 24.00         1
 23.00         1
-8.00          1
 0.05          1
 0.21          1
 18.00         1
-18.00         1
-0.70          1
 17.70         1
-17.00         1
-0.20          1
-6.00          1
-0.90          1
 1.30          1
Name: 구매수량, dtype: int64

In [22]:
len(dt1[(dt1['구매수량'] == 0) & (dt1['구매금액'] == 0)])

25

In [17]:
# 구매수량, 구매금액 모두 0인거 제거
idx = dt1[(dt1['구매수량'] == 0.0) & (dt1['구매금액'] == 0.0)].index
dt1_unique = dt1.drop(idx)
dt1_unique.reset_index(inplace = True)

In [29]:
dt1_unique.index

RangeIndex(start=0, stop=123166, step=1)

In [18]:
print(len(dt1))
print(len(dt1_unique))

123191
123166


In [3]:
# 고객데이터에 오전, 오후, 저녁 방문율 계산해서 넣을 변수 생성

mfile['오전방문율'] = 0
mfile['오후방문율'] = 0
mfile['저녁방문율'] = 0

In [42]:
visit_rate = dt1[['공급일자','회원번호','구매시각']]
visit_rate.head()

,공급일자,회원번호,구매시각
0,2018-01-02,272369856,10:04
1,2018-01-02,1476143616,10:09
2,2018-01-02,743192512,10:12
3,2018-01-02,176912064,10:23
4,2018-01-02,87592064,10:29


In [24]:
len(visit_rate)

123166

In [43]:
# 공급일자, 회원번호, 구매시각이 같은 중복행 제거하고 유일값만 남김
unique_visit_rate = visit_rate.drop_duplicates()
unique_visit_rate.reset_index(inplace = True)
len(unique_visit_rate)

70277

# plus

## 1분 사이로 구매한 사람도 거르기 위해 시 단위로 중복행 제거

In [61]:
unique_visit_rate.columns

Index(['index', '공급일자', '회원번호', '구매시각', '오전방문', '오후방문', '저녁방문', '시', '오전오후저녁'], dtype='object')

In [62]:
uni_hour = unique_visit_rate[['공급일자','회원번호','오전오후저녁']]
uni_hour_2 = uni_hour.drop_duplicates()
len(uni_hour_2)

67365

In [67]:
unique_visit_rate[(unique_visit_rate['오전방문'] + unique_visit_rate['오후방문'] + unique_visit_rate['저녁방문']) > 0]

,index,공급일자,회원번호,구매시각,오전방문,오후방문,저녁방문,시,오전오후저녁
0,0,2018-01-02,272369856,10:04,1,0,0,10,오전
1,1,2018-01-02,1476143616,10:09,1,0,0,10,오전
2,2,2018-01-02,743192512,10:12,1,0,0,10,오전
3,3,2018-01-02,176912064,10:23,1,0,0,10,오전
4,4,2018-01-02,87592064,10:29,1,0,0,10,오전
...,...,...,...,...,...,...,...,...,...
70272,123178,2018-10-31,436681536,12:03,0,1,0,12,오후
70273,123179,2018-10-31,56570880,12:42,0,1,0,12,오후
70274,123180,2018-10-31,1566343360,13:03,0,1,0,13,오후
70275,123185,2018-10-31,467498560,16:33,0,1,0,16,오후


In [68]:
unique_visit_rate.drop('index', axis = 1, inplace = True)


/home/pirl/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [44]:
unique_visit_rate['오전방문'] = 0
unique_visit_rate['오후방문'] = 0
unique_visit_rate['저녁방문'] = 0

unique_visit_rate['시'] = 0

for i in range(len(unique_visit_rate)):
    unique_visit_rate['시'][i] = int(unique_visit_rate['구매시각'][i].split(':')[0])
unique_visit_rate['시'].value_counts()

/home/pirl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/pirl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/pirl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

10    8978
16    8202
11    7630
15    7256
17    7201
12    7101
13    6870
14    6573
18    6080
19    4212
20     114
9       57
8        3
Name: 시, dtype: int64

In [45]:
unique_visit_rate['오전오후저녁'] = ''


/home/pirl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


## '오전오후저녁' 변수에 할당

In [49]:
unique_visit_rate.loc[unique_visit_rate['시'] < 12, '오전오후저녁'] = '오전'
unique_visit_rate.loc[(unique_visit_rate['시'] >= 12) & (unique_visit_rate['시'] < 17), '오전오후저녁'] = '오후'
unique_visit_rate.loc[unique_visit_rate['시'] >= 17, '오전오후저녁'] = '저녁'

# 오전, 오후, 저녁 더미변수로 생성(방문 : 1, 미방문: 0)

In [65]:
unique_visit_rate.loc[unique_visit_rate['시'] < 12, '오전방문'] = 1
unique_visit_rate.loc[(unique_visit_rate['시'] >= 12) & (unique_visit_rate['시'] < 17), '오후방문'] = 1
unique_visit_rate.loc[unique_visit_rate['시'] >= 17, '저녁방문'] = 1

/home/pirl/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [50]:
unique_visit_rate

,index,공급일자,회원번호,구매시각,오전방문,오후방문,저녁방문,시,오전오후저녁
0,0,2018-01-02,272369856,10:04,0,0,0,10,오전
1,1,2018-01-02,1476143616,10:09,0,0,0,10,오전
2,2,2018-01-02,743192512,10:12,0,0,0,10,오전
3,3,2018-01-02,176912064,10:23,0,0,0,10,오전
4,4,2018-01-02,87592064,10:29,0,0,0,10,오전
...,...,...,...,...,...,...,...,...,...
70272,123178,2018-10-31,436681536,12:03,0,0,0,12,오후
70273,123179,2018-10-31,56570880,12:42,0,0,0,12,오후
70274,123180,2018-10-31,1566343360,13:03,0,0,0,13,오후
70275,123185,2018-10-31,467498560,16:33,0,0,0,16,오후


In [52]:
u = unique_visit_rate.groupby(['회원번호','오전오후저녁'])['오전오후저녁'].count()
u_stack = u.unstack()

In [54]:
u_stack.fillna(0, inplace= True)
u_stack

오전오후저녁,오전,오후,저녁
회원번호,,,
18613824,0.0,1.0,0.0
18792000,0.0,1.0,0.0
18942336,0.0,0.0,2.0
18949760,0.0,1.0,0.0
19627200,13.0,28.0,25.0
...,...,...,...
1643209600,0.0,2.0,0.0
1643337664,0.0,0.0,1.0
1643489856,0.0,1.0,0.0


# dt1 매장 시간대별 방문율

In [72]:
u_stack['총방문수'] = u_stack['오전'] + u_stack['오후'] + u_stack['저녁']
u_stack['오전방문율'] = round(u_stack['오전'] / u_stack['총방문수'],2)
u_stack['오후방문율'] = round(u_stack['오후'] / u_stack['총방문수'],2)
u_stack['저녁방문율'] = round(u_stack['저녁'] / u_stack['총방문수'],2)
u_stack

오전오후저녁,오전,오후,저녁,총방문수,오전방문율,오후방문율,저녁방문율
회원번호,,,,,,,
18613824,0.0,1.0,0.0,1.0,0.00,1.00,0.00
18792000,0.0,1.0,0.0,1.0,0.00,1.00,0.00
18942336,0.0,0.0,2.0,2.0,0.00,0.00,1.00
18949760,0.0,1.0,0.0,1.0,0.00,1.00,0.00
19627200,13.0,28.0,25.0,66.0,0.20,0.42,0.38
...,...,...,...,...,...,...,...
1643209600,0.0,2.0,0.0,2.0,0.00,1.00,0.00
1643337664,0.0,0.0,1.0,1.0,0.00,0.00,1.00
1643489856,0.0,1.0,0.0,1.0,0.00,1.00,0.00


In [78]:
visit_rate_final = pd.DataFrame({'회원번호' : u_stack.index, '오전' : u_stack['오전']}, index = np.array(range(5518)))

In [79]:
visit_rate_final = u_stack


,회원번호,오전
0,18613824,NaN
1,18792000,NaN
2,18942336,NaN
3,18949760,NaN
4,19627200,NaN
...,...,...
5513,1643209600,NaN
5514,1643337664,NaN
5515,1643489856,NaN
5516,1670400000,NaN


In [81]:
u_stack['오전']

회원번호
18613824       0.0
18792000       0.0
18942336       0.0
18949760       0.0
19627200      13.0
              ... 
1643209600     0.0
1643337664     0.0
1643489856     0.0
1670400000    51.0
1670606016     1.0
Name: 오전, Length: 5518, dtype: float64

In [84]:
dt1_1=dt1.copy()
dt1_1.drop(79299,axis=0,inplace=True)#구매수량 -인데 구매이력이 없고 구매금액이 0인것 제거
#구매수량,구매금액 모두 0인거 제거;바코드만 찍고 안살경우니까
dt1_1.drop(17988,axis=0,inplace=True)
dt1_1.drop(109050,axis=0,inplace=True)
dt1_1.reset_index(drop=True,inplace=True)

In [87]:


visit_rate = dt1_1[['공급일자','회원번호','구매시각']]
visit_rate.head()

visit_rate['시'] = 0

# 구매시각에서 시간만 추출
for i in range(len(visit_rate)):
    visit_rate['시'][i] = int(visit_rate['구매시각'][i].split(':')[0])
# visit_rate['시'].value_counts()

# 공급일자, 회원번호, 시가 같은 중복행 제거하고 유일값만 남김
visit_rate.drop('구매시각', axis = 1, inplace = True)
unique_visit_rate = visit_rate.drop_duplicates()
unique_visit_rate.reset_index(inplace = True)

# 오전, 오후, 저녁방문여부 더미변수 생성
unique_visit_rate['오전방문'] = 0
unique_visit_rate['오후방문'] = 0
unique_visit_rate['저녁방문'] = 0
unique_visit_rate.loc[unique_visit_rate['시'] < 12, '오전방문'] = 1
unique_visit_rate.loc[(unique_visit_rate['시'] >= 12) & (unique_visit_rate['시'] < 17), '오후방문'] = 1
unique_visit_rate.loc[unique_visit_rate['시'] >= 17, '저녁방문'] = 1

# 오전오후저녁 변수 생성
unique_visit_rate['오전오후저녁'] = ''
unique_visit_rate.loc[unique_visit_rate['시'] < 12, '오전오후저녁'] = '오전'
unique_visit_rate.loc[(unique_visit_rate['시'] >= 12) & (unique_visit_rate['시'] < 17), '오전오후저녁'] = '오후'
unique_visit_rate.loc[unique_visit_rate['시'] >= 17, '오전오후저녁'] = '저녁'

# 회원번호의 오전, 오후, 저녁별 방문수 count
u = unique_visit_rate.groupby(['회원번호','오전오후저녁'])['오전오후저녁'].count()
u_stack = u.unstack()
u_stack.fillna(0, inplace= True)

# 한 지점의 회원별 총방문수와 오전, 오후, 저녁 방문율 계산
u_stack['총방문수'] = u_stack['오전'] + u_stack['오후'] + u_stack['저녁']
u_stack['오전방문율'] = round(u_stack['오전'] / u_stack['총방문수'],2)
u_stack['오후방문율'] = round(u_stack['오후'] / u_stack['총방문수'],2)
u_stack['저녁방문율'] = round(u_stack['저녁'] / u_stack['총방문수'],2)
u_stack

/home/pirl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/pirl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/pirl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/panda

오전오후저녁,오전,오후,저녁,총방문수,오전방문율,오후방문율,저녁방문율
회원번호,,,,,,,
18613824,0.0,1.0,0.0,1.0,0.00,1.00,0.00
18792000,0.0,1.0,0.0,1.0,0.00,1.00,0.00
18942336,0.0,0.0,2.0,2.0,0.00,0.00,1.00
18949760,0.0,1.0,0.0,1.0,0.00,1.00,0.00
19627200,12.0,26.0,23.0,61.0,0.20,0.43,0.38
...,...,...,...,...,...,...,...
1643209600,0.0,2.0,0.0,2.0,0.00,1.00,0.00
1643337664,0.0,0.0,1.0,1.0,0.00,0.00,1.00
1643489856,0.0,1.0,0.0,1.0,0.00,1.00,0.00


# dt1_1 방문율 추가

In [100]:
dt1_1=dt1.copy()
dt1_1.drop(79299,axis=0,inplace=True)#구매수량 -인데 구매이력이 없고 구매금액이 0인것 제거
#구매수량,구매금액 모두 0인거 제거;바코드만 찍고 안살경우니까
dt1_1.drop(17988,axis=0,inplace=True)
dt1_1.drop(109050,axis=0,inplace=True)
dt1_1.reset_index(drop=True,inplace=True)

In [101]:
# dt1_1방문율 데이터 만들기

visit_rate = dt1_1[['공급일자','회원번호','구매시각']]
visit_rate.head()

visit_rate['시'] = 0

# 구매시각에서 시간만 추출
for i in range(len(visit_rate)):
    visit_rate['시'][i] = int(visit_rate['구매시각'][i].split(':')[0])
# visit_rate['시'].value_counts()

# 공급일자, 회원번호, 시가 같은 중복행 제거하고 유일값만 남김
visit_rate.drop('구매시각', axis = 1, inplace = True)
unique_visit_rate = visit_rate.drop_duplicates()
unique_visit_rate.reset_index(inplace = True)

# 오전, 오후, 저녁방문여부 더미변수 생성
unique_visit_rate['오전방문'] = 0
unique_visit_rate['오후방문'] = 0
unique_visit_rate['저녁방문'] = 0
unique_visit_rate.loc[unique_visit_rate['시'] < 12, '오전방문'] = 1
unique_visit_rate.loc[(unique_visit_rate['시'] >= 12) & (unique_visit_rate['시'] < 17), '오후방문'] = 1
unique_visit_rate.loc[unique_visit_rate['시'] >= 17, '저녁방문'] = 1

# 오전오후저녁 변수 생성
unique_visit_rate['오전오후저녁'] = ''
unique_visit_rate.loc[unique_visit_rate['시'] < 12, '오전오후저녁'] = '오전'
unique_visit_rate.loc[(unique_visit_rate['시'] >= 12) & (unique_visit_rate['시'] < 17), '오전오후저녁'] = '오후'
unique_visit_rate.loc[unique_visit_rate['시'] >= 17, '오전오후저녁'] = '저녁'

# 회원번호의 오전, 오후, 저녁별 방문수 count
u = unique_visit_rate.groupby(['회원번호','오전오후저녁'])['오전오후저녁'].count()
u_stack = u.unstack()
u_stack.fillna(0, inplace= True)

# 한 지점의 회원별 총방문수와 오전, 오후, 저녁 방문율 계산
u_stack['총방문수'] = u_stack['오전'] + u_stack['오후'] + u_stack['저녁']
u_stack['오전방문율'] = round(u_stack['오전'] / u_stack['총방문수'],2)
u_stack['오후방문율'] = round(u_stack['오후'] / u_stack['총방문수'],2)
u_stack['저녁방문율'] = round(u_stack['저녁'] / u_stack['총방문수'],2)
u_stack.to_csv('dt1_visit_rate.csv', encoding = 'utf-8')

/home/pirl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/pirl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/home/pirl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.p

# dt4_1 방문율 추가

In [102]:
dt4_1 = dt4.copy()
dt4_1.drop([25602, 51584], axis = 0, inplace = True)  # 구매 없이 환불된 기록 누락
dt4_1.loc[51513,'반품_원거래일자'] = 20181025 # 반품 날짜 오류 수정
dt4_1.reset_index(drop=True,inplace=True) # 인덱스 재조정

In [104]:
# dt4_1방문율 데이터 만들기

visit_rate = dt4_1[['공급일자','회원번호','구매시각']]
visit_rate.head()

visit_rate['시'] = 0

# 구매시각에서 시간만 추출
for i in range(len(visit_rate)):
    visit_rate['시'][i] = int(visit_rate['구매시각'][i].split(':')[0])
# visit_rate['시'].value_counts()

# 공급일자, 회원번호, 시가 같은 중복행 제거하고 유일값만 남김
visit_rate.drop('구매시각', axis = 1, inplace = True)
unique_visit_rate = visit_rate.drop_duplicates()
unique_visit_rate.reset_index(inplace = True)

# 오전, 오후, 저녁방문여부 더미변수 생성
unique_visit_rate['오전방문'] = 0
unique_visit_rate['오후방문'] = 0
unique_visit_rate['저녁방문'] = 0
unique_visit_rate.loc[unique_visit_rate['시'] < 12, '오전방문'] = 1
unique_visit_rate.loc[(unique_visit_rate['시'] >= 12) & (unique_visit_rate['시'] < 17), '오후방문'] = 1
unique_visit_rate.loc[unique_visit_rate['시'] >= 17, '저녁방문'] = 1

# 오전오후저녁 변수 생성
unique_visit_rate['오전오후저녁'] = ''
unique_visit_rate.loc[unique_visit_rate['시'] < 12, '오전오후저녁'] = '오전'
unique_visit_rate.loc[(unique_visit_rate['시'] >= 12) & (unique_visit_rate['시'] < 17), '오전오후저녁'] = '오후'
unique_visit_rate.loc[unique_visit_rate['시'] >= 17, '오전오후저녁'] = '저녁'

# 회원번호의 오전, 오후, 저녁별 방문수 count
u = unique_visit_rate.groupby(['회원번호','오전오후저녁'])['오전오후저녁'].count()
u_stack = u.unstack()
u_stack.fillna(0, inplace= True)

# 한 지점의 회원별 총방문수와 오전, 오후, 저녁 방문율 계산
u_stack['총방문수'] = u_stack['오전'] + u_stack['오후'] + u_stack['저녁']
u_stack['오전방문율'] = round(u_stack['오전'] / u_stack['총방문수'],2)
u_stack['오후방문율'] = round(u_stack['오후'] / u_stack['총방문수'],2)
u_stack['저녁방문율'] = round(u_stack['저녁'] / u_stack['총방문수'],2)
u_stack.to_csv('dt4_visit_rate.csv', encoding = 'utf-8')

/home/pirl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/pirl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/home/pirl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.p

# dt2 방문율 추가

In [105]:
# dt2방문율 데이터 만들기

visit_rate = dt2[['공급일자','회원번호','구매시각']]
visit_rate.head()

visit_rate['시'] = 0

# 구매시각에서 시간만 추출
for i in range(len(visit_rate)):
    visit_rate['시'][i] = int(visit_rate['구매시각'][i].split(':')[0])
# visit_rate['시'].value_counts()

# 공급일자, 회원번호, 시가 같은 중복행 제거하고 유일값만 남김
visit_rate.drop('구매시각', axis = 1, inplace = True)
unique_visit_rate = visit_rate.drop_duplicates()
unique_visit_rate.reset_index(inplace = True)

# 오전, 오후, 저녁방문여부 더미변수 생성
unique_visit_rate['오전방문'] = 0
unique_visit_rate['오후방문'] = 0
unique_visit_rate['저녁방문'] = 0
unique_visit_rate.loc[unique_visit_rate['시'] < 12, '오전방문'] = 1
unique_visit_rate.loc[(unique_visit_rate['시'] >= 12) & (unique_visit_rate['시'] < 17), '오후방문'] = 1
unique_visit_rate.loc[unique_visit_rate['시'] >= 17, '저녁방문'] = 1

# 오전오후저녁 변수 생성
unique_visit_rate['오전오후저녁'] = ''
unique_visit_rate.loc[unique_visit_rate['시'] < 12, '오전오후저녁'] = '오전'
unique_visit_rate.loc[(unique_visit_rate['시'] >= 12) & (unique_visit_rate['시'] < 17), '오전오후저녁'] = '오후'
unique_visit_rate.loc[unique_visit_rate['시'] >= 17, '오전오후저녁'] = '저녁'

# 회원번호의 오전, 오후, 저녁별 방문수 count
u = unique_visit_rate.groupby(['회원번호','오전오후저녁'])['오전오후저녁'].count()
u_stack = u.unstack()
u_stack.fillna(0, inplace= True)

# 한 지점의 회원별 총방문수와 오전, 오후, 저녁 방문율 계산
u_stack['총방문수'] = u_stack['오전'] + u_stack['오후'] + u_stack['저녁']
u_stack['오전방문율'] = round(u_stack['오전'] / u_stack['총방문수'],2)
u_stack['오후방문율'] = round(u_stack['오후'] / u_stack['총방문수'],2)
u_stack['저녁방문율'] = round(u_stack['저녁'] / u_stack['총방문수'],2)
u_stack.to_csv('dt2_visit_rate.csv', encoding = 'utf-8')

/home/pirl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/pirl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/home/pirl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.p

# dt3 방문율 만들기

In [106]:
# dt3방문율 데이터 만들기

visit_rate = dt3[['공급일자','회원번호','구매시각']]
visit_rate.head()

visit_rate['시'] = 0

# 구매시각에서 시간만 추출
for i in range(len(visit_rate)):
    visit_rate['시'][i] = int(visit_rate['구매시각'][i].split(':')[0])
# visit_rate['시'].value_counts()

# 공급일자, 회원번호, 시가 같은 중복행 제거하고 유일값만 남김
visit_rate.drop('구매시각', axis = 1, inplace = True)
unique_visit_rate = visit_rate.drop_duplicates()
unique_visit_rate.reset_index(inplace = True)

# 오전, 오후, 저녁방문여부 더미변수 생성
unique_visit_rate['오전방문'] = 0
unique_visit_rate['오후방문'] = 0
unique_visit_rate['저녁방문'] = 0
unique_visit_rate.loc[unique_visit_rate['시'] < 12, '오전방문'] = 1
unique_visit_rate.loc[(unique_visit_rate['시'] >= 12) & (unique_visit_rate['시'] < 17), '오후방문'] = 1
unique_visit_rate.loc[unique_visit_rate['시'] >= 17, '저녁방문'] = 1

# 오전오후저녁 변수 생성
unique_visit_rate['오전오후저녁'] = ''
unique_visit_rate.loc[unique_visit_rate['시'] < 12, '오전오후저녁'] = '오전'
unique_visit_rate.loc[(unique_visit_rate['시'] >= 12) & (unique_visit_rate['시'] < 17), '오전오후저녁'] = '오후'
unique_visit_rate.loc[unique_visit_rate['시'] >= 17, '오전오후저녁'] = '저녁'

# 회원번호의 오전, 오후, 저녁별 방문수 count
u = unique_visit_rate.groupby(['회원번호','오전오후저녁'])['오전오후저녁'].count()
u_stack = u.unstack()
u_stack.fillna(0, inplace= True)

# 한 지점의 회원별 총방문수와 오전, 오후, 저녁 방문율 계산
u_stack['총방문수'] = u_stack['오전'] + u_stack['오후'] + u_stack['저녁']
u_stack['오전방문율'] = round(u_stack['오전'] / u_stack['총방문수'],2)
u_stack['오후방문율'] = round(u_stack['오후'] / u_stack['총방문수'],2)
u_stack['저녁방문율'] = round(u_stack['저녁'] / u_stack['총방문수'],2)
u_stack.to_csv('dt3_visit_rate.csv', encoding = 'utf-8')

/home/pirl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/pirl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/home/pirl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.p

# dt1_1, dt4_1.csv 데이터

In [107]:
dt1_1.to_csv('dt1_1.csv', encoding = 'utf-8')
dt4_1.to_csv('dt4_1.csv', encoding = 'utf-8')